In [1]:
import gensim
import numpy as np
import string
import tensorflow as tf
import Placeholders
import datetime
import random as random

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
with tf.Session() as sess:
    x = tf.add(3,4).eval()
    print(x)

7


In [3]:
x = Placeholders.sentence
y_ = Placeholders.class_labels
c = Placeholders.class_centroids

In [4]:
def get_cluster_based_labels(fc_layer_representation):
    indexed_centroids = tf.matmul(y_,c)
    x_distances = tf.exp(-(tf.square(fc_layer_representation - indexed_centroids)))
    sum_x_distances = tf.log(np.sum(x_distances))
    logits = tf.divide(x_distances, sum_x_distances)
    return logits
    

In [5]:
def one_hot(vec, vals=Placeholders.n_classes):
    n = vec.shape[1]
    print ("len",n)
    out = np.zeros((n, vals))
    out[np.arange(n), vec] = 1
    return out

In [6]:
def train(sess):
    hidden1 =tf.layers.dense(Placeholders.sentence, Placeholders.hidden1_neurons)
    hidden2 =tf.layers.dense(Placeholders.sentence, Placeholders.hidden2_neurons)
    ##Basic Neural Network
    #cluster_based_labels = hidden2
    #Cluster based objective
    cluster_based_labels = get_cluster_based_labels(hidden2)
    logits = tf.layers.dense(cluster_based_labels, Placeholders.n_classes)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits,
                                                                   labels=Placeholders.class_labels))
    loss = tf.reduce_mean(cross_entropy)
    train_step = tf.train.AdamOptimizer(1e-5).minimize(loss)

    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Placeholders.class_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    STEPS = 3000
    MINIBATCH_SIZE = 20
    
    print ("Starting at:" , datetime.datetime.now())
    sess.run(tf.global_variables_initializer())
    print ("Initialization done at:" , datetime.datetime.now())
    
    n_samples = 100
    mu, sigma = 10, 10
    X = np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])
    Y = np.zeros(n_samples, dtype=np.int16)
    mu, sigma = 50, 10
    X = np.vstack((X, np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])))
    Y = np.vstack((Y,np.ones(n_samples,dtype=np.int16))).reshape(-1,n_samples *2 )
    print(Y)
    print (X.shape,Y.shape)
    Y = one_hot(Y)
    print (X.shape,Y.shape)
    C = np.ones(Placeholders.hidden2_neurons)*10
    C = np.vstack((C,np.ones(Placeholders.hidden2_neurons)*50)).reshape(-1,Placeholders.hidden2_neurons)
    
    #sess.run(train_step, feed_dict={x: X, y_: Y, c:C})
    
    for i in range(STEPS):
        random_index = random.randint(0,n_samples * 2 - MINIBATCH_SIZE - 1)
        random_X = X[random_index + MINIBATCH_SIZE,:].reshape(-1, Placeholders.word_vec_dim)
        random_Y = Y[random_index + MINIBATCH_SIZE,:].reshape(-1, Placeholders.n_classes)
        print(random_X.shape)
        sess.run(train_step, feed_dict={x: random_X, y_: random_Y, c:C})
        acc = sess.run(accuracy, feed_dict={x: X, y_: Y, c:C})*100
        print (acc)

    

In [7]:
with tf.Session() as sess:
    train(sess)

Starting at: 2018-02-10 20:23:58.472813
Initialization done at: 2018-02-10 20:23:58.621239
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
(200, 300) (1, 200)
len 200
(200, 300) (200, 2)
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0

50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(

50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(

50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(

50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(1, 300)
50.0
(